# Deploy model as a webservice on Azure Container Instance

## Table of contents
1. [Prerequisites](#prerequisites)

2. [Initialize workspace](#workspace)

3. [Deploy Model to ACI](#deploymodel)

- a) [Create scoring file](#scoringfile)
- b) [Define Enviroment](#env)
- c) [Deployment configuration](#configfile)
- d  [Deploy Webservice](#webservice)
- e) [Test Webservice](#testservice)



### 1. Prerequisites <a id='prerequisites'></a>

In [ ]:
import numpy as np 
import azureml.core

# display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

### 2. Initialize workspace <a id='workspace'></a>

In [103]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

In [ ]:
subscription_id = "4a571c1c-a483-4a43-9930-490479d70db0"
resource_group = "Learn_MLOps"
workspace = "MLOs_WS"

In [ ]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

### 3. Deploy model <a id='deploymodel'></a>

#### a) Create a scoring script <a id='scoringfile'></a>

In [ ]:
scaler = ml_client.models.download(name='scaler',version='1',download_path='./model')

In [ ]:
svc_model = ml_client.models.download(name='support-vector-classifier',version='1', download_path='./model')

In [ ]:
%%writefile score.py
import json
import logging
import numpy as np
import os
import pickle
import joblib
import onnxruntime
import time

def init():
    global model, scaler, input_name, label_name
    # , inputs_dc, prediction_dc
    

    scaler_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model/scaler/scaler.pkl')
    # deserialize the model file back into a sklearn model
    scaler = joblib.load(scaler_path)
    
    model_onnx = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model/support-vector-classifier/svc.onnx')
    # print(os.listdir(model_onnx))
    model = onnxruntime.InferenceSession(model_onnx, None)
    input_name = model.get_inputs()[0].name
    label_name = model.get_outputs()[0].name
    
 
def run(data):
    try: 
        data = json.loads(data)["data"]
        data = np.array(data)
        data = scaler.fit_transform(data.reshape(1, 6))

        # model inference
        result = model.run([label_name], {input_name: data.astype(np.float32)})[0]
        # this call is saving model output data into Azure Blob

    except Exception as e:   
        result = 'error'
        print(e)

    return result.tolist()       


### Define the endpoint and deployment

In [104]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

# online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")
online_endpoint_name = 'nerdward-endpoint-1'

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a sample online endpoint",
    auth_mode="key",
    tags={"foo": "bar"},
)

In [105]:
ml_client.online_endpoints.begin_create_or_update(endpoint)

In [106]:
model1 = Model(name='weather-aci-prediction',path='./model')

In [111]:
# model2 = Model(name='support-vector-classifier')
env = Environment(
    conda_file="./model/conda.yml",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model1,
    environment=env,
    code_configuration=CodeConfiguration(
        code="./", scoring_script="score.py"
    ),
    instance_type="Standard_F2s_v2",
    instance_count=1,
)

In [112]:
ml_client.online_deployments.begin_create_or_update(blue_deployment)

Check: endpoint nerdward-endpoint-1 exists
Uploading 06_Model_Deployment (0.43 MBs): 100%|██████████| 427361/427361 [00:00<00:00, 6978874.15it/s]


data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


...........................................

In [116]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint)

In [ ]:
ml_client.online_endpoints.get(name=online_endpoint_name)

In [118]:
print(ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=online_endpoint_name, lines=50
))

Instance status:
SystemSetup: Succeeded
UserContainerImagePull: Succeeded
ModelDownload: Succeeded
UserContainerStart: Succeeded

Container events:
Kind: Pod, Name: Pulling, Type: Normal, Time: 2023-01-03T15:47:19.621374Z, Message: Start pulling container image
Kind: Pod, Name: Downloading, Type: Normal, Time: 2023-01-03T15:47:23.295755Z, Message: Start downloading models
Kind: Pod, Name: Pulled, Type: Normal, Time: 2023-01-03T15:48:27.029718Z, Message: Container image is pulled successfully
Kind: Pod, Name: Downloaded, Type: Normal, Time: 2023-01-03T15:48:27.029718Z, Message: Models are downloaded successfully
Kind: Pod, Name: Created, Type: Normal, Time: 2023-01-03T15:48:27.317847Z, Message: Created container inference-server
Kind: Pod, Name: Started, Type: Normal, Time: 2023-01-03T15:48:27.428383Z, Message: Started container inference-server
Kind: Pod, Name: ContainerReady, Type: Normal, Time: 2023-01-03T15:48:44.88018746Z, Message: Container is ready

Container logs:
Werkzeug==2.2.

In [ ]:
print("Kind\tLocation\tName")
print("-------\t----------\t------------------------")
for endpoint in ml_client.online_endpoints.list():
    print(f"{endpoint.kind}\t{endpoint.location}\t{endpoint.name}")

In [117]:
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="./model/sample-request.json",
)

'[1]'

In [ ]:
import json


input_payload = json.dumps({
    'data': [[34.927778, 0.24, 7.3899, 83, 16.1000, 1]]
})

with open('./model/sample-request.json', 'w') as f:
    f.write(input_payload)

In [119]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)

...................................................................................................

#### b) Define Environment <a id='env'></a>

In [ ]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

Environment(name="myenv")

#env = Environment.get(workspace=ws, name="AzureML-Minimal")
env = Environment.get(workspace=ws, name="AzureML-Minimal").clone('myenv')



In [ ]:
for pip_package in ["numpy", "onnxruntime", "joblib", "azureml-core", "azureml-monitoring", "azureml-defaults", "scikit-learn==0.20.3", "inference-schema", "inference-schema[numpy-support]"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='score.py',
                                    environment=env)

#### c) Deployment Configuration <a id='configfile'></a>

In [ ]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, collect_model_data=True)

#### d) Deploy web service <a id='webservice'></a>

In [ ]:
model1 = Model(ws, 'model-scaler')
model2 = Model(ws, 'support-vector-classifier')

service_name = 'weather-aci-prediction'

In [ ]:
service = Model.deploy(ws, service_name, models=[model1, model2], inference_config=inference_config, deployment_config=deployment_config, overwrite=True)
service.wait_for_deployment(show_output = True)
print(service.state)

In [ ]:
print(service.get_logs())

In [ ]:
service.update(enable_app_insights=True)

#### e) Test web service <a id='testservice'></a>

In [ ]:
print(service.scoring_uri)

In [ ]:
print(service.swagger_uri)

In [ ]:
service.state

In [ ]:
import json


input_payload = json.dumps({
    'data': [[34.927778, 0.24, 7.3899, 83, 16.1000, 1016.51, 1]],
    'method': 'predict'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)

In [ ]:
# service.delete()